In [1]:
!pip install langchain rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!pip install openai tiktoken langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:
import os 

os.environ["OPENAI_API_KEY"] = "enter your OPENAI_API_KEY"

### Data

In [3]:
from langchain.document_loaders import UnstructuredPDFLoader

file_path = "/kaggle/input/ragdata/Orca_paper.pdf"
documents = UnstructuredPDFLoader(
    file_path
)

docs = documents.load()

In [4]:
print(docs[0].page_content)

3 2 0 2

n u J

5

] L C . s c [

1 v 7 0 7 2 0 . 6 0 3 2 : v i X r a

Orca: Progressive Learning from Complex

Explanation Traces of GPT-4

Subhabrata Mukherjee∗†, Arindam Mitra∗

Ganesh Jawahar, Sahaj Agarwal, Hamid Palangi, Ahmed Awadallah

Microsoft Research

Abstract

Recent research has focused on enhancing the capability of smaller models through imitation learning, drawing on the outputs generated by large foundation models (LFMs). A number of issues impact the quality of these models, ranging from limited imitation signals from shallow LFM outputs; small scale homogeneous training data; and most notably a lack of rigorous evaluation resulting in overestimating the small model’s capability as they tend to learn to imitate the style, but not the reasoning process of LFMs. To address these challenges, we develop Orca, a 13-billion parameter model that learns to imitate the reasoning process of LFMs. Orca learns from rich signals from GPT-4 including explanation traces; step-by-st

### Split data

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

chunks = splitter.split_documents(docs)

In [7]:
chunks[0].page_content

'3 2 0 2\n\nn u J\n\n5\n\n] L C . s c [\n\n1 v 7 0 7 2 0 . 6 0 3 2 : v i X r a\n\nOrca: Progressive Learning from Complex\n\nExplanation Traces of GPT-4\n\nSubhabrata Mukherjee∗†, Arindam Mitra∗\n\nGanesh Jawahar, Sahaj Agarwal, Hamid Palangi, Ahmed Awadallah\n\nMicrosoft Research\n\nAbstract'

### Embeddings and Indexing

In [13]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq

embeddings = OpenAIEmbeddings()

llm = ChatGroq(
    groq_api_key = "enter your GROQ_API_KEY",
    model_name = 'llama3-70b-8192'
)

vectordb = Chroma.from_documents(
    chunks,
    embeddings
)

In [14]:
vectordb_retreiver = vectordb.as_retriever(
    search_kwargs = {"k": 3}
)

In [15]:
# BM25 Retriever
from langchain.retrievers import BM25Retriever, EnsembleRetriever

keyword_retriever = BM25Retriever.from_documents(
    chunks
)
keyword_retriever.k = 3

### Ensamble Retriever

In [16]:
ensemble_retriever = EnsembleRetriever(
    retrievers = [vectordb_retreiver, keyword_retriever],
    weights = [0.7, 0.3] # must sum to 1
)

### prompt tempelate

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following quesion based on the retrieved context.
    Think step by step.
    You will get a reward if your answers were helpful.
    <context>
    {context}
    </context>
    
    Question: {query}
    """
)
output_parser = StrOutputParser()

### Chain 

In [18]:
chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [19]:
print(chain.invoke("What is instruction tuning?"))

According to the context, instruction tuning is a technique that allows pre-trained language models to learn from input (natural language descriptions of the task) and response pairs. It has been applied to both language-only and multimodal tasks, and has been shown to improve the zero-shot and few-shot performance of models such as FLAN and InstructGPT on various benchmarks.


In [20]:
print(chain.invoke("How does Orca compares to ChatGPT?"))

Based on the provided context, here's a step-by-step analysis of how Orca compares to ChatGPT:

1. **Table Understanding**: ChatGPT has better table understanding and reasoning capabilities than Orca, with Orca lagging behind by 7.4% in the penguins in a table task.

2. **Multilingual Understanding**: Orca performs on par with GPT-4, exceeding ChatGPT by 4.7%. Both Orca and ChatGPT achieve parity on the salient translation error detection task.

3. **Logical and Geometric Reasoning**: ChatGPT shows superior logical reasoning capabilities compared to Orca, outperforming Orca by at least 9% in the Boolean expressions and Web of Lies tasks. However, Orca performs better than ChatGPT in the logical deduction task for five objects, but ChatGPT excels in the three and seven objects tasks, outperforming Orca by at least 4.9%. ChatGPT also has better geometric reasoning capabilities, outperforming Orca by 23% in the geometric shape task.

In summary, Orca has strengths in multilingual understa